### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [2]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    reviews_new=reviews.sort_values(order_by)
    training_df = reviews_new.head(training_size)
    validation_df = reviews_new.iloc[training_size:training_size+testing_size]
    
    return training_df, validation_df

In [3]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [4]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [5]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [6]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.608391
2 		 5.948475
3 		 4.149649
4 		 3.099513
5 		 2.411851
6 		 1.928044
7 		 1.570826
8 		 1.298113
9 		 1.084847
10 		 0.915023
11 		 0.777857
12 		 0.665774
13 		 0.573271
14 		 0.496258
15 		 0.431633
16 		 0.377015
17 		 0.330555
18 		 0.290804
19 		 0.256614
20 		 0.227072
21 		 0.201437
22 		 0.179111
23 		 0.159601
24 		 0.142501
25 		 0.127471
26 		 0.114228
27 		 0.102534
28 		 0.092185
29 		 0.083009
30 		 0.074857
31 		 0.067604
32 		 0.061140
33 		 0.055369
34 		 0.050210
35 		 0.045591
36 		 0.041449
37 		 0.037730
38 		 0.034387
39 		 0.031377
40 		 0.028663
41 		 0.026214
42 		 0.024001
43 		 0.021998
44 		 0.020184
45 		 0.018538
46 		 0.017044
47 		 0.015685
48 		 0.014448
49 		 0.013322
50 		 0.012294
51 		 0.011356
52 		 0.010498
53 		 0.009714
54 		 0.008995
55 		 0.008336
56 		 0.007732
57 		 0.007177
58 		 0.006667
59 		 0.006197
60 		 0.005765
61 		 0.005366
62 		 0.004998
63 		 0.004659
64 		 

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [7]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Use the training data to create a series of users and movies that matches the ordering in training data
    user_ids_series = np.array(train_data_df.index)
    movie_ids_series = np.array(train_data_df.columns)
    
    # User row and Movie Column
    user_row = np.where(user_ids_series == user_id)[0][0]
    movie_col = np.where(movie_ids_series == movie_id)[0][0]
    
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(user_matrix[user_row,:], movie_matrix[:,movie_col])
   
    return pred

In [8]:
train_data_df.loc[8, 2844]

nan

In [31]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

6.8740573989166949

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [10]:
train_data_df.iloc[0,1]

nan

In [11]:
train_data_df.index[0]

8

In [12]:
movies.head(2)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [13]:
str(movies[movies['movie_id']==8]['movie'])

'0    Edison Kinetoscopic Record of a Sneeze (1894)\nName: movie, dtype: object'

In [14]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    user_row = np.where(train_data_df.index == user_id)[0][0]
    movie_col = np.where(train_data_df.columns == movie_id)[0][0]
    original = train_data_df.iloc[user_row, movie_col]
    
    movie_name = str(movies[movies['movie_id']==movie_id]['movie'])[5:]
    movie_name = movie_name.replace('\nName: movie, dtype: object', '')
    
    print("For user {} we predict a {} rating for the movie {}, rating {}.".
          format(user_id, round(prediction, 2), str(movie_name), original))
   
    
    return prediction


In [15]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

For user 8 we predict a 6.87 rating for the movie  Fantômas - À l'ombre de la guillotine (1913), rating nan.


6.8740573989166949

Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [17]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    val_users = np.array(val_df['user_id'])
    val_movies = np.array(val_df['movie_id'])
    val_ratings = np.array(val_df['rating'])
    
    # Perform the predicted vs. actual for the first 6 rows.  How does it look?
    for idx in range(num_preds):
        pred = predict_rating(user_mat, movie_mat, val_users[idx], val_movies[idx])
        print("The actual rating for user {} on movie {} is {}.\n While the predicted rating is {}.".format(val_users[idx], val_movies[idx], val_ratings[idx], round(pred))) 


In [22]:
# Perform the predicted vs. actual for the first 6 rows.  What happened?
validation_comparison(val_df, 6)     

The actual rating for user 49056 on movie 1598822 is 8.
 While the predicted rating is 6.0.
The actual rating for user 49056 on movie 289879 is 9.
 While the predicted rating is 9.0.
The actual rating for user 49056 on movie 1563738 is 9.
 While the predicted rating is 8.0.
The actual rating for user 49056 on movie 1458175 is 4.
 While the predicted rating is 7.0.
The actual rating for user 28599 on movie 103639 is 8.
 While the predicted rating is 7.0.
The actual rating for user 50593 on movie 1560985 is 4.
 While the predicted rating is 3.0.


In [23]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

The actual rating for user 49056 on movie 1598822 is 8.
 While the predicted rating is 6.0.
The actual rating for user 49056 on movie 289879 is 9.
 While the predicted rating is 9.0.
The actual rating for user 49056 on movie 1563738 is 9.
 While the predicted rating is 8.0.
The actual rating for user 49056 on movie 1458175 is 4.
 While the predicted rating is 7.0.
The actual rating for user 28599 on movie 103639 is 8.
 While the predicted rating is 7.0.
The actual rating for user 50593 on movie 1560985 is 4.
 While the predicted rating is 3.0.


IndexError: index 0 is out of bounds for axis 0 with size 0


** A statement about why you think what happened happened.**

In [25]:
val_users = np.array(val_df['user_id'])
val_movies = np.array(val_df['movie_id'])
val_ratings = np.array(val_df['rating'])

In [26]:
idx=6
print(val_users[idx], val_movies[idx], val_ratings[idx])

29000 287978 9


In [36]:
idx=6
user_id=val_users[idx]
movie_id=val_movies[idx]

# Get user and movie lists in training data set
user_ids_series = np.array(train_data_df.index)
movie_ids_series = np.array(train_data_df.columns)
    
# User row and Movie Column
print(np.where(user_ids_series == user_id))
print(np.where(movie_ids_series == movie_id))

(array([1739]),)
(array([], dtype=int64),)


There is no movie_id = 287978 in the training data set. So using training data set can't predict on the validation data.